# Fine-Tune de um LLM para sumarização de diálogo

Neste notebook, você ajustará um LLM existente do Hugging Face para um resumo aprimorado do diálogo. Você usará o modelo [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5), que fornece um modelo ajustado para instruções de alta qualidade e pode resumir o texto imediatamente. 

# Índice

- [ 1 - Carregar dependências necessárias, conjunto de dados e LLM](#1)
  - [ 1.1 - Carregar conjunto de dados e LLM](#1.1)
  - [ 1.2 - Teste o modelo com inferência zero shot](#1.2)
- [ 2 - Execute o ajuste fino completo](#2)
  - [ 2.1 - Pré-processar o conjunto de dados de resumo de diálogo](#2.1)
  - [ 2.2 - Ajuste o modelo com o conjunto de dados pré-processado](#2.2)
  - [ 2.3 - Avalie o modelo qualitativamente (avaliação humana)](#2.3)

<a name='1'></a>
## 1 - Carregar dependências necessárias, conjunto de dados e LLM

<a name='1.1'></a>
### 1.1 - Carregar dependências necessárias

In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 --quiet

Importe os componentes necessários. 

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import pandas as pd
import numpy as np

<a name='1.1'></a>
### 1.2 - Carregar conjunto de dados e LLM

Você continuará experimentando o conjunto de dados [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) Hugging Face. Ele contém mais de 10.000 diálogos com resumos e tópicos rotulados manualmente.

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

Carregue o [modelo FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) pré-treinado e seu tokenizer diretamente do HuggingFace. Observe que você usará a [versão pequena](https://huggingface.co/google/flan-t5-base) do FLAN-T5. A configuração `torch_dtype=torch.bfloat16` especifica o tipo de memória a ser usada por este modelo.

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

É possível extrair a quantidade de parâmetros do modelo e descobrir quantos deles são treináveis. A função a seguir pode ser usada para fazer isso.

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

<a name='1.2'></a>
### 1.2 - Teste o modelo com inferência zero shot

Teste o modelo com a inferência zero hhot. Você pode ver que o modelo tem dificuldade para resumir o diálogo em comparação com o resumo feito por um humano, mas extrai algumas informações importantes do texto que indicam que o modelo pode ser ajustado para a tarefa em questão.

In [ ]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'PROMPT DE ENTRADA:\n{prompt}')
print(dash_line)
print(f'SUMÁRIO HUMANO:\n{summary}\n')
print(dash_line)
print(f'GERADO PELO MODELO - ZERO SHOT:\n{output}')

<a name='2'></a>
## 2 - Execute o ajuste fino completo

<a name='2.1'></a>
### 2.1 - Pré-processar o conjunto de dados de resumo de diálogo

Você precisa converter os pares diálogo-resumo (prompt-response) em instruções explícitas para o LLM:

Prompt de treinamento (diálogo):
```
Summarize the following conversation.

    Palla: This is his part of the conversation.
    Ana: This is her part of the conversation.
    
Summary: 
```

Resposta do treinamento (resumo):
```
Both Palla and Ana participated in the conversation.
```

Em seguida, pré-processe o conjunto de dados de prompt-response em tokens e extraia seus `input_ids` (1 por token)

In [ ]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# O conjunto de dados contém, na verdade, 3 divisões de comparação: treinamento, validação, teste.
# O código tokenize_function está manipulando todos os dados em todas as divisões em lotes.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Para economizar algum tempo, você utilizará uma subamostra do conjunto de dados:

In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Verifique as formas de todas as três partes do conjunto de dados:

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

O conjunto de dados de saída está pronto para ajuste fino.

<a name='2.2'></a>
### 2.2 - Ajuste o modelo com o conjunto de dados pré-processado

Agora utilize a classe Hugging Face `Trainer` integrada (veja a documentação [aqui](https://huggingface.co/docs/transformers/main_classes/trainer)). Passe o conjunto de dados pré-processado com referência ao modelo original. 

In [ ]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

Treinar uma versão totalmente ajustada do modelo levaria algumas horas em uma GPU. Para economizar tempo, baixe um ponto de verificação do modelo totalmente ajustado para usar no restante deste notebook. Esse modelo totalmente ajustado também será chamado de **modelo de instrução** neste laboratório.

In [ ]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint/

O tamanho do modelo de instrução baixado é de aproximadamente 1 GB.

In [ ]:
!ls -alh ./flan-dialogue-summary-checkpoint/pytorch_model.bin

Crie uma instância da classe `AutoModelForSeq2SeqLM` para o modelo de instrução:

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype=torch.bfloat16)

<a name='2.3'></a>
### 2.3 - Avalie o modelo qualitativamente (avaliação humana)

Tal como acontece com muitos aplicativos GenAI, uma abordagem qualitativa em que você se pergunta "Meu modelo está se comportando da maneira que deveria?" geralmente é um bom ponto de partida. No exemplo abaixo (o mesmo com o qual iniciamos este notebook), você pode ver como o modelo ajustado é capaz de criar um resumo razoável do diálogo em comparação com a incapacidade original de entender o que está sendo pedido ao modelo.

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'SUMÁRIO HUMANO:\n{human_baseline_summary}')
print(dash_line)
print(f'MODELO ORIGINAL:\n{original_model_text_output}')
print(dash_line)
print(f'MODELO DE INSTRUÇÃO:\n{instruct_model_text_output}')